# 28. mtplib - 파일 첨부하여 이메일 보내기

- smtplib 모듈은 이메일을 송신할때 사용하는 모듈이다.  

- 다음과 같은 정보를 기준으로 코드를 작성.
```
SMTP 메일서버 주소 : "smtp.gmail.com" (gmail의 SMTP 서버주소)
보내는 메일 계정 : "ironmanciti@gmail.com" (본인의 메일계정으로 사용할 것)
보내는 메일 계정의 비밀번호 : xxxxxxx (본인의 비밀번호로 사용할 것)
받는 사람의 메일 계정 : "trimurti@naver.com" (발송한 메일을 확인할 수 있는 메일계정)
메일의 제목 : 파일첨부 메일송신 테스트
메일의 내용 : 첨부된 파일 2개를 확인해 주세요.
보내는 파일 2개의 위치 (jpg, pdf 등)
```

- email 모듈을 이용하여 발송할 메시지를 작성한다. 메시지는 파일을 첨부해야 하기 때문에 MIMEMultipart 클래스를 사용해야 한다.

In [2]:
import datetime
import re
import pandas as pd
import smtplib, ssl

from email.utils import formatdate
from email.mime.multipart import MIMEMultipart
from email.header import Header

# 메일 제목과 내용을 설정하는 모듈
from email.mime.text import MIMEText
import os
from email.mime.base import MIMEBase
from email.encoders import encode_base64

In [3]:
msg = MIMEMultipart()

생성한 메시지 객체에 보내는 사람과 받는 사람을 설정  

만약 수신인이 한 명이 아닌 여러명일 경우 다음과 같이 콤마+스페이스로 구분하여 여러명을 추가.
```
msg['To'] = 'pahkey@gmail.com, admin@wikidocs.net'
```

In [4]:
# 날짜 설정
formatdate(localtime=True)

'Wed, 07 Sep 2022 14:01:07 +0900'

In [5]:
port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = "ironmanciti@gmail.com"  # 보내는 사람 주소
receiver_email = "trimurti@naver.com"  # 받는 사람 주소
with open("password.txt", "r") as pswd:  # 앱 비밀번호
    password = pswd.read()

FileNotFoundError: [Errno 2] No such file or directory: 'password.txt'

message 구성

In [10]:
msg['From'] = sender_email 
msg['To'] = receiver_email 
msg['Date'] = formatdate(localtime=True)

제목은 같이 Header 클래스를 사용하여 설정한다. Header 클래스를 사용해야 한글이 깨지는 인코딩 문제가 발생하지 않는다. 

In [11]:
msg['Subject'] = Header(s='파일첨부 메일송신 테스트', charset='utf-8')

메시지의 본문 내용은 MIMEText를 사용하여 작성.

만들어진 body를 메시지(MIMEMultipart)에 추가

In [12]:
body = """
            첨부된 파일2개를 확인해 주세요. <br>
            그리고, <br>
            홈페이지 링크 함께 보냅니다. <br>
            <a href="https://ironmanciti.github.io/">오영제의 홈페이지입니다.</a>
"""

msg.attach(MIMEText(body, "html"))

메일에 첨부할 파일 리스트를 작성

In [13]:
files = list()
files.append('winequality-red.csv')
files.append('innerJoin.png')

- MIMEBase 클래스를 이용하여 생성한 파일내용을 메시지에 추가  
    - `application`, `octet-stream` : 이진 파일을 보낼 때 사용. 이진 파일이면 pdf 던 엑셀이던 모두 가능. 옥텟 스트림은 이름에서처럼 8비트 단위의 바이너리 데이터를 의미  
    - set_payload 함수를 이용하여 첨부하고 싶은 파일을 읽어 옴  
    
- encode_base64 함수는 추가한 파일을 base64로 인코딩하는 역할을 한다.

In [14]:
for f in files:
    part = MIMEBase('application', "octet-stream")
    part.set_payload(open(f,"rb").read())
    encode_base64(part)
    part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(f))
    msg.attach(part)

작성한 메시지를 SMTP 서버를 사용하여 발송한다. 다음과 같이 SMTP_SSL 객체를 생성한다.

In [15]:
context = ssl.create_default_context()

with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, msg.as_string())   
    server.quit()

- email이 잘 도착 되었는지 확인

## 위에서 작성한 내용을 하나의 함수로 결합

In [16]:
%%writefile email_sender.py
def send_email(subject, body, files=[]):
    import datetime
    import re
    import pandas as pd
    import smtplib, ssl

    from email.utils import formatdate
    from email.mime.multipart import MIMEMultipart
    from email.header import Header
    # 메일 제목과 내용을 설정하는 모듈
    from email.mime.text import MIMEText
    import os
    from email.mime.base import MIMEBase
    from email.encoders import encode_base64
    
    msg = MIMEMultipart()

    port = 465  # For SSL
    smtp_server = "smtp.gmail.com"
    sender_email = "ironmanciti@gmail.com"  # Enter your address
    receiver_email = "trimurti@naver.com"  # Enter receiver address
    with open("password.txt", "r") as pswd:
        password = pswd.read()

    msg['From'] = sender_email 
    msg['To'] = receiver_email 
    msg['Date'] = formatdate(localtime=True)

    msg['Subject'] = Header(s=subject, charset='utf-8')
    body = MIMEText(body, _charset='utf-8')
    msg.attach(body)
    
    for f in files:
        part = MIMEBase('application', "octet-stream")
        part.set_payload(open(f,"rb").read())
        encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(f))
        msg.attach(part)
        
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())   
        server.quit()
        
    print("성공적으로 메일 발송")

Overwriting email_sender.py


In [18]:
from email_sender import send_email

subject = '파일첨부 메일송신 테스트'
body = '첨부된 파일 2개를 확인해 주세요.\n다음 주에 식사 같이 하십시다.'
files = list()
files.append('winequality-red.csv')
files.append('16_index.html')

send_email(subject, body, files)

성공적으로 메일 발송
